In [57]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
batch_size = 4
max_iters = 10000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250


cuda


In [58]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [59]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,
         0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0,
         1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25,
        38, 28,  1, 39, 30,  1, 39, 50,  9,  1])


In [60]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
tensor([[73, 61, 58, 66,  9,  1, 72, 68],
        [67, 62, 67, 60,  1, 65, 58, 54],
        [62, 56, 61,  1, 56, 68, 67, 72],
        [78,  1, 61, 54, 75, 58,  1, 78]], device='cuda:0')
targets:
tensor([[61, 58, 66,  9,  1, 72, 68, 66],
        [62, 67, 60,  1, 65, 58, 54, 57],
        [56, 61,  1, 56, 68, 67, 72, 73],
        [ 1, 61, 54, 75, 58,  1, 78, 68]], device='cuda:0')


In [61]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [62]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)



A8 K7G25E4q5N*eec3ZBGG;DS_M(D!&436(CGRyYKo-tH:['1(szpL8zLLn9sq
RKPw*43!wS: 85EfVclxB8bHrNEgVsev ,357GG2]!gSJ9Rj'M"jJA2dW7OOlL;Xov3FcrO4kKG76qVYMr"8
59zdjZ,_A.,﻿8wDiH8?RKj"f[aQN*;c;gpHbmlFTJw&("xN!Cdbd6SW8?eip5[oe(q
kP"﻿or9,DFP"i&*.57:RaOkP-TH!C(YR[b)t0LMQM:a&(3b.au2F*Csg5Eng[2)HBHbTTI-MXBOKjC6J9Y'8ng47G]GqAJ59)Qds
2
u&'V?e a'mN!vgP-*TdblarIO3blxWlvY4GiX &CkdDX;nxN?bo-rT;VuGD?Muy:!?[uy8Icw1gmHiCvpihTSBM3P"Y4ZcmQ:a&7OdNz"QM"Y4L&HQ:Tbu;sN[SF79xBzRKu.ni)1WIb). neI[BG]﻿D!FKVaVW ;wEVW8R8L5h
;5mQc_q4tJ


In [63]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.828, val loss: 4.831
step: 250, train loss: 4.807, val loss: 4.758
step: 500, train loss: 4.721, val loss: 4.708
step: 750, train loss: 4.644, val loss: 4.652
step: 1000, train loss: 4.595, val loss: 4.591
step: 1250, train loss: 4.523, val loss: 4.523
step: 1500, train loss: 4.466, val loss: 4.472
step: 1750, train loss: 4.422, val loss: 4.426
step: 2000, train loss: 4.384, val loss: 4.374
step: 2250, train loss: 4.298, val loss: 4.325
step: 2500, train loss: 4.247, val loss: 4.259
step: 2750, train loss: 4.209, val loss: 4.203
step: 3000, train loss: 4.155, val loss: 4.155
step: 3250, train loss: 4.106, val loss: 4.107
step: 3500, train loss: 4.057, val loss: 4.060
step: 3750, train loss: 4.026, val loss: 4.042
step: 4000, train loss: 3.956, val loss: 3.969
step: 4250, train loss: 3.926, val loss: 3.945
step: 4500, train loss: 3.892, val loss: 3.894
step: 4750, train loss: 3.848, val loss: 3.867
step: 5000, train loss: 3.790, val loss: 3.810
step: 5250, train l

In [55]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


;
_  AZ3DFRx, 3tMO-?Y7bS0﻿hqkpncwbrZiC.MS'y_Rsasn'[lt2TOkF(]k]isw344fiSFj'v)?m"*Hr59﻿zy*s﻿")j"d
bUr;5oaMZ6l1uZPar)QDYcI!A(10.doq7(x*H7hq*RB,;. irVU)O!, b4"
dl. Rs-w0

mAa0'zgK&:2LCaj"t&)V)ZikutyM0x_OK&5﻿hfR]*0
sMKs_E"i
1rNf,NOzvFB,xKlS;eMiyj5Hrhm;3JtA, &*lC7FAUJtHphqX)Qt:ckC;P1ulkces2BqlJ;?y50'pYp?)fhK1irrrbl"DIDK(0*tq;vJR"W;PDK_aduyG
s5EI6bUnJ'KjO7K AlSKcUn Iv,9BuyR6;0)OOpTof1, EzrA5Pk&ZZEt4Nse-'BG&myJR:RBwWXcrp*4QL*&5:aTm5,UX]?FVUOpIrhqht1A,jDOZc4outhL.:Pa.hJP&4Hma]iteyq;Lira:Z19y(nGmx"N
